In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd


In [8]:
from pathlib import Path

dataset_root = Path("../dataset")
dataset_root.mkdir(exist_ok=True)

env_metadata = pd.read_excel(dataset_root / "environment_metadata.xlsx").set_index("Unnamed: 0")
trajectory_metadata = pd.read_excel(dataset_root / "trajectory_metadata.xlsx")

trajectories = np.load(dataset_root / "camera_trajectory_data.npy")
ratings = np.load(dataset_root / "ratings" / "observer1.npy", allow_pickle=True)


In [9]:
goal_positions = np.load(dataset_root / "environment_0" / "camera_cube_position.npy")

Setup of cost model

In [10]:
joint_position = tf.keras.Input(shape=(2))
goal = tf.keras.Input(shape=(2))

output = tf.linalg.norm(goal - joint_position, axis=-1)

model = tf.keras.Model((joint_position, goal), output)

In [31]:
def score(trajectories):
    trajectories = trajectories.reshape((-1, 2))
    result = np.empty((trajectories.shape[0], len(goal_positions)), dtype=np.float_)

    for idx, goal in enumerate(goal_positions):
        goal_vector = np.empty((len(trajectories), 2), dtype=np.float_)
        goal_vector[:] = goal
        result[:, idx] = model.predict((trajectories, goal_vector))
    return result.reshape((-1, 330, len(goal_positions)))

In [32]:
foo = score(trajectories)

In [35]:
np.allclose(foo[0], ratings[0])

True